# LLM-as-a-Judge Tutorial: Multi-Turn Conversation Evaluation with MLflow 3.7

This interactive notebook demonstrates how to use MLflow 3.7's session-level evaluation features to assess multi-turn customer support conversations.

## Tutorial Goals

1. Track multi-turn conversations with session IDs using MLflow tracing
2. Create session-level judges with `{{ conversation }}` template
3. Evaluate conversation quality (coherence and context retention)
4. Demonstrate MLflow 3.7's new session tracking features

## Scenario

A customer support agent handles multi-turn conversations. Session-level judges evaluate:
- **Coherence**: Does the conversation flow logically across turns?
- **Context Retention**: Does the agent remember information from earlier messages?

**Key MLflow 3.7 Feature:**

The `{{ conversation }}` template variable in judge instructions automatically makes judges session-level aware. MLflow aggregates all traces with the same session ID into a single conversation view for evaluation.

---

Based on the [customer_support_agent.py](customer_support_agent.py) CLI pattern, adapted for interactive notebook tutorial.

## Setup: Import Dependencies

First, let's import all necessary libraries and load environment variables.

In [ ]:
from genai.common.config import AgentConfig
from genai.agents.multi_turn.customer_support_agent_cls import CustomerSupportAgent
from genai.agents.multi_turn.scenarios import (
    get_scenario_printer_troubleshooting,
    get_scenario_account_access
)
from genai.agents.multi_turn.prompts import (
    get_coherence_judge_instructions,
    get_context_retention_judge_instructions,
    map_retention_to_score
)
from genai.common.mlflow_config import setup_mlflow_tracking
import mlflow
from typing import List, Dict, Any
import os
from pathlib import Path
import pandas as pd

# Load environment variables from .env file if it exists
try:
    from dotenv import load_dotenv
    env_file = Path(".env")
    if env_file.exists():
        load_dotenv(env_file)
        print(f"✓ Loaded environment variables from {env_file.absolute()}")
    else:
        print(f"ℹ️  No .env file found at {env_file.absolute()}")
        print("   You can create one with your credentials or set environment variables manually")
except ImportError:
    print("ℹ️  python-dotenv not installed. Install with: pip install python-dotenv")
    print("   Or set environment variables manually in the cells below")

## Configuration: Set Up Environment

**Two ways to configure credentials:**

1. **Recommended**: Create a `.env` file in this directory with your credentials
2. **Alternative**: Uncomment and set credentials in the cells below

### Create a `.env` file (Recommended)

Create a file named `.env` in the same directory as this notebook:

**For Databricks:**
```
DATABRICKS_TOKEN=your-token-here
DATABRICKS_HOST=https://your-workspace.cloud.databricks.com
```

**For OpenAI:**
```
OPENAI_API_KEY=sk-your-key-here
```

The cell above will automatically load these credentials.

In [ ]:
# ============================================================================
# CONFIGURATION: Choose your provider and models
# ============================================================================

# Option 1: Databricks (default)
PROVIDER = "databricks"
AGENT_MODEL = "databricks-gpt-5"
JUDGE_MODEL = "databricks-gemini-2-5-flash"

# Option 2: OpenAI (uncomment to use)
# PROVIDER = "openai"
# AGENT_MODEL = "gpt-4o-mini"
# JUDGE_MODEL = "gpt-4o"

# Other settings
TEMPERATURE = 1.0
EXPERIMENT_NAME = "customer-support-agent-notebook"

print("✓ Configuration set:")
print(f"  Provider: {PROVIDER}")
print(f"  Agent Model: {AGENT_MODEL}")
print(f"  Judge Model: {JUDGE_MODEL}")

### Verify Credentials (Optional Manual Setup)

If you didn't create a `.env` file, you can set credentials manually by uncommenting the appropriate lines below:

In [ ]:
# ============================================================================
# MANUAL CREDENTIAL SETUP (if not using .env file)
# ============================================================================

# For Databricks - Uncomment and set these if you didn't create a .env file
# os.environ["DATABRICKS_TOKEN"] = "your-token-here"
# os.environ["DATABRICKS_HOST"] = "https://your-workspace.cloud.databricks.com"

# For OpenAI - Uncomment and set this if you didn't create a .env file
# os.environ["OPENAI_API_KEY"] = "sk-your-key-here"

# Verify credentials are set
if PROVIDER == "databricks":
    if "DATABRICKS_TOKEN" in os.environ and "DATABRICKS_HOST" in os.environ:
        print("✓ Databricks credentials found")
    else:
        print("⚠️  Missing Databricks credentials!")
        print("   Create a .env file or uncomment the lines above to set credentials")
elif PROVIDER == "openai":
    if "OPENAI_API_KEY" in os.environ:
        print("✓ OpenAI credentials found")
    else:
        print("⚠️  Missing OpenAI credentials!")
        print("   Create a .env file or uncomment the lines above to set credentials")

## Step 1: Setup MLflow Tracing

Enable MLflow tracing to capture all customer support conversations and evaluation results automatically.

In [ ]:
setup_mlflow_tracking(
    experiment_name=EXPERIMENT_NAME,
    enable_autolog=True
)

print("✓ MLflow tracing enabled")
print(f"  Experiment: {EXPERIMENT_NAME}")
print("  View traces: mlflow ui")

## Step 2: Import the Agent Class

Instead of redefining the class, we import it directly from the module to follow DRY principles.

In [ ]:
# CustomerSupportAgent already imported at top of notebook
print("✓ CustomerSupportAgent imported successfully")
print("\nThe class provides:")
print("  - handle_message(): Process single message with session tracking")
print("  - run_conversation(): Execute complete multi-turn conversation")
print("  - evaluate_session(): Judge conversation quality using MLflow 3.7")

## Step 2 (continued): Initialize Agent and Judges

Create the configuration and instantiate our customer support agent with session-level judges.

In [ ]:
# Create agent configuration
config = AgentConfig(
    model=AGENT_MODEL,
    provider=PROVIDER,
    temperature=TEMPERATURE,
    mlflow_experiment=EXPERIMENT_NAME
)

# Initialize the agent with judges
agent = CustomerSupportAgent(config, judge_model=JUDGE_MODEL, debug=False)

print("✓ Agent and Judges initialized")
print(f"  Provider: {config.provider}")
print(f"  Agent Model: {config.model}")
print(f"  Judge Model: {JUDGE_MODEL}")
print(f"  Temperature: {config.temperature}")

## Understanding Session Tracking

Before running scenarios, let's understand how MLflow 3.7's session tracking works.

### Multi-Turn Session Tracking Flow

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    MULTI-TURN SESSION TRACKING FLOW                      │
│                          (MLflow 3.7 Features)                           │
└─────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────┐
│  CONVERSATION SESSION: "session-printer-001"                            │
│  (Single logical conversation = Multiple MLflow traces)                 │
└─────────────────────────────────────────────────────────────────────────┘
                                    │
                                    ▼
        ╔═══════════════════════════════════════════════════════╗
        ║  TURN 1: User sends first message                     ║
        ║  "My HP LaserJet 3000 won't turn on"                  ║
        ╚═══════════════════════════════════════════════════════╝
                                    │
                                    ▼
        ┌───────────────────────────────────────────────────────┐
        │  @mlflow.trace() decorator creates Trace #1          │
        │  trace_id: tr-abc123...                               │
        ├───────────────────────────────────────────────────────┤
        │  mlflow.update_current_trace(                         │
        │      metadata={                                       │
        │          "mlflow.trace.session": "session-printer-001"│
        │      }                                                 │
        │  )                                                     │
        ├───────────────────────────────────────────────────────┤
        │  TRACE DATA:                                          │
        │    - Input: "My HP LaserJet 3000 won't turn on"       │
        │    - Output: "Let me help. Is power cable connected?" │
        │    - Session Tag: "session-printer-001" ✓             │
        │    - Timestamp: 2025-12-07 10:00:01                   │
        └───────────────────────────────────────────────────────┘
                                    │
                                    ▼
        ╔═══════════════════════════════════════════════════════╗
        ║  TURN 2: User responds                                ║
        ║  "Yes, power cable is plugged in securely"            ║
        ╚═══════════════════════════════════════════════════════╝
                                    │
                                    ▼
        ┌───────────────────────────────────────────────────────┐
        │  @mlflow.trace() creates NEW Trace #2                │
        │  trace_id: tr-def456...                               │
        ├───────────────────────────────────────────────────────┤
        │  mlflow.update_current_trace(                         │
        │      metadata={                                       │
        │          "mlflow.trace.session": "session-printer-001"│
        │      }                                                 │
        │  )                                                     │
        ├───────────────────────────────────────────────────────┤
        │  TRACE DATA:                                          │
        │    - Input: "Yes, power cable is plugged in..."       │
        │    - Output: "Try different outlet in another room"   │
        │    - Session Tag: "session-printer-001" ✓             │
        │    - Timestamp: 2025-12-07 10:00:45                   │
        └───────────────────────────────────────────────────────┘
                                    │
                                    ▼
        ╔═══════════════════════════════════════════════════════╗
        ║  TURN 3: User tries solution                          ║
        ║  "Tried different outlet, still nothing"              ║
        ╚═══════════════════════════════════════════════════════╝
                                    │
                                    ▼
        ┌───────────────────────────────────────────────────────┐
        │  @mlflow.trace() creates NEW Trace #3                │
        │  trace_id: tr-ghi789...                               │
        ├───────────────────────────────────────────────────────┤
        │  mlflow.update_current_trace(                         │
        │      metadata={                                       │
        │          "mlflow.trace.session": "session-printer-001"│
        │      }                                                 │
        │  )                                                     │
        ├───────────────────────────────────────────────────────┤
        │  TRACE DATA:                                          │
        │    - Input: "Tried different outlet, still nothing"   │
        │    - Output: "Contact HP for warranty replacement"    │
        │    - Session Tag: "session-printer-001" ✓             │
        │    - Timestamp: 2025-12-07 10:02:10                   │
        └───────────────────────────────────────────────────────┘
                                    │
                                    ▼
┌─────────────────────────────────────────────────────────────────────────┐
│                      SESSION-LEVEL EVALUATION                            │
│                      (MLflow 3.7 Key Feature)                           │
└─────────────────────────────────────────────────────────────────────────┘
                                    │
                                    ▼
        ┌───────────────────────────────────────────────────────┐
        │  STEP 1: Search for all traces in session            │
        │                                                       │
        │  mlflow.search_traces(                                │
        │      experiment_ids=[exp_id],                         │
        │      filter_string="run_id = 'run-xyz'"               │
        │  )                                                     │
        │                                                       │
        │  FINDS: [Trace #1, Trace #2, Trace #3]               │
        │  All tagged with "session-printer-001"                │
        └───────────────────────────────────────────────────────┘
                                    │
                                    ▼
        ┌───────────────────────────────────────────────────────┐
        │  STEP 2: Aggregate traces with {{ conversation }}    │
        │                                                       │
        │  Judge instructions contain:                          │
        │  "Evaluate this conversation: {{ conversation }}"     │
        │                                                       │
        │  MLflow replaces {{ conversation }} with:             │
        │  ┌────────────────────────────────────────────────┐  │
        │  │ User: My HP LaserJet 3000 won't turn on       │  │
        │  │ Assistant: Let me help. Is power cable...     │  │
        │  │ User: Yes, power cable is plugged in...       │  │
        │  │ Assistant: Try different outlet in another... │  │
        │  │ User: Tried different outlet, still nothing   │  │
        │  │ Assistant: Contact HP for warranty...         │  │
        │  └────────────────────────────────────────────────┘  │
        └───────────────────────────────────────────────────────┘
                                    │
                                    ▼
        ┌───────────────────────────────────────────────────────┐
        │  STEP 3: Session-Level Judges Evaluate                │
        │                                                       │
        │  ┌─────────────────────────────────────────────────┐ │
        │  │ COHERENCE JUDGE (Boolean)                       │ │
        │  │ - Reads entire conversation                     │ │
        │  │ - Checks logical flow across turns              │ │
        │  │ - Output: True (coherent)                       │ │
        │  │ - Rationale: "Conversation flows naturally..."  │ │
        │  └─────────────────────────────────────────────────┘ │
        │                                                       │
        │  ┌─────────────────────────────────────────────────┐ │
        │  │ CONTEXT RETENTION JUDGE (Categorical)           │ │
        │  │ - Reads entire conversation                     │ │
        │  │ - Checks if agent remembers context             │ │
        │  │ - Output: "excellent"                           │ │
        │  │ - Rationale: "Agent perfectly recalls HP..."    │ │
        │  └─────────────────────────────────────────────────┘ │
        └───────────────────────────────────────────────────────┘
                                    │
                                    ▼
        ╔═══════════════════════════════════════════════════════╗
        ║  FINAL RESULTS                                        ║
        ║  ✓ Coherence: PASS (True)                            ║
        ║  ✓ Context Retention: EXCELLENT (4/4)                ║
        ╚═══════════════════════════════════════════════════════╝

┌─────────────────────────────────────────────────────────────────────────┐
│                           KEY DIFFERENCES                                │
└─────────────────────────────────────────────────────────────────────────┘

  TURN-LEVEL EVALUATION          │  SESSION-LEVEL EVALUATION
  (Traditional approach)          │  (MLflow 3.7 with {{ conversation }})
─────────────────────────────────┼─────────────────────────────────────────
  1 trace = 1 evaluation          │  N traces = 1 evaluation
  Evaluates individual responses  │  Evaluates entire conversation
  No context between turns        │  Full conversation context
  Can't assess coherence          │  Can assess flow and coherence
  Can't assess retention          │  Can assess memory/retention
  3 traces → 3 judge calls        │  3 traces → 1 judge call (aggregated)
```

## Step 3: First Scenario - Printer Troubleshooting

Load the printer troubleshooting scenario from the pre-defined scenarios.

In [ ]:
# Load first scenario
scenario1 = get_scenario_printer_troubleshooting()

print(f"Scenario: {scenario1['name']}")
print(f"Description: {scenario1['description']}")
print(f"Session ID: {scenario1['session_id']}")
print(f"Number of turns: {len(scenario1['messages'])}")
print(f"\nMessages:")
for i, msg in enumerate(scenario1['messages'], 1):
    print(f"  Turn {i}: {msg[:60]}..." if len(msg) > 60 else f"  Turn {i}: {msg}")

## Step 4: Run Multi-Turn Conversation

Execute the conversation and see the agent respond to each turn.

In [ ]:
# Start MLflow run for this conversation
with mlflow.start_run(run_name=scenario1['name']) as run:
    # Run the conversation
    conv_result1 = agent.run_conversation(
        messages=scenario1['messages'],
        session_id=scenario1['session_id']
    )
    
    # Store run ID for evaluation
    run_id_1 = run.info.run_id
    
print(f"\n✓ Conversation completed with {conv_result1['turns']} turns")
print(f"  Session ID: {conv_result1['session_id']}")
print(f"  Run ID: {run_id_1}")

## Step 5: Evaluate Session with Session-Level Judges

Now evaluate the entire conversation using MLflow 3.7's session-level evaluation.

In [ ]:
print(f"Evaluating session '{scenario1['session_id']}'...\n")

try:
    eval_result1 = agent.evaluate_session(scenario1['session_id'], run_id_1)
    
    # Display results
    print(f"{'='*70}")
    print(f"Evaluation Results: {scenario1['name']}")
    print(f"{'='*70}")
    
    # Coherence results
    coherence_symbol = "✓" if eval_result1['coherence']['passed'] else "✗"
    coherence_label = "PASS" if eval_result1['coherence']['passed'] else "FAIL"
    print(f"\n📊 Coherence: {coherence_symbol} {coherence_label}")
    print(f"   Value: {eval_result1['coherence']['feedback_value']}")
    print(f"   Rationale: {eval_result1['coherence']['rationale']}")
    
    # Context retention results
    retention_value = str(eval_result1['context_retention']['feedback_value']).upper()
    retention_score = eval_result1['context_retention']['score']
    print(f"\n🧠 Context Retention: {retention_value}")
    print(f"   Score: {retention_score}/4")
    print(f"   Rationale: {eval_result1['context_retention']['rationale']}")
    
    print(f"\n{'='*70}\n")
    
except Exception as e:
    print(f"\n✗ Evaluation failed: {e}")
    print(f"Error details: {type(e).__name__}")
    import traceback
    traceback.print_exc()
    print("\nEnsure MLflow >= 3.7.0 for {{ conversation }} template support.\n")

## Step 6: Second Scenario - Account Access

Run the account access scenario to compare evaluation results.

In [ ]:
# Load second scenario
scenario2 = get_scenario_account_access()

print(f"{'─'*70}")
print(f"Scenario: {scenario2['name']}")
print(f"Description: {scenario2['description']}")
print(f"Session ID: {scenario2['session_id']}")
print(f"{'─'*70}\n")

# Start MLflow run for this conversation
with mlflow.start_run(run_name=scenario2['name']) as run:
    # Run conversation
    conv_result2 = agent.run_conversation(
        messages=scenario2['messages'],
        session_id=scenario2['session_id']
    )
    
    run_id_2 = run.info.run_id
    
    print(f"\nEvaluating session '{scenario2['session_id']}'...\n")
    
    try:
        eval_result2 = agent.evaluate_session(scenario2['session_id'], run_id_2)
        
        # Display results
        print(f"{'='*70}")
        print(f"Evaluation Results: {scenario2['name']}")
        print(f"{'='*70}")
        
        # Coherence results
        coherence_symbol = "✓" if eval_result2['coherence']['passed'] else "✗"
        coherence_label = "PASS" if eval_result2['coherence']['passed'] else "FAIL"
        print(f"\n📊 Coherence: {coherence_symbol} {coherence_label}")
        print(f"   Value: {eval_result2['coherence']['feedback_value']}")
        print(f"   Rationale: {eval_result2['coherence']['rationale']}")
        
        # Context retention results
        retention_value = str(eval_result2['context_retention']['feedback_value']).upper()
        retention_score = eval_result2['context_retention']['score']
        print(f"\n🧠 Context Retention: {retention_value}")
        print(f"   Score: {retention_score}/4")
        print(f"   Rationale: {eval_result2['context_retention']['rationale']}")
        
        print(f"\n{'='*70}\n")
        
    except Exception as e:
        print(f"\n✗ Evaluation failed: {e}")
        import traceback
        traceback.print_exc()

## Debug Mode: Understanding DataFrame Structure

Enable debug mode to see MLflow's internal DataFrame structure and how evaluation results are stored.

In [ ]:
# Re-initialize agent with debug mode
agent_debug = CustomerSupportAgent(config, judge_model=JUDGE_MODEL, debug=True)

print("✓ Agent re-initialized with debug=True")
print("\nRe-running evaluation of Scenario 1 with debug output...\n")

try:
    eval_result_debug = agent_debug.evaluate_session(scenario1['session_id'], run_id_1)
    print("\n✓ Debug evaluation completed")
    print("\nThe debug output above shows:")
    print("  - Available DataFrame columns")
    print("  - DataFrame shape (rows x columns)")
    print("  - Coherence and context retention column names")
    print("  - Raw values extracted from judges")
except Exception as e:
    print(f"\n✗ Debug evaluation failed: {e}")

## Additional Scenarios: Billing Inquiry

Let's test with a billing support conversation that should demonstrate excellent context retention.

In [ ]:
# Define billing inquiry scenario
scenario_billing = {
    "name": "Billing Inquiry",
    "session_id": "session-billing-003",
    "description": "Customer questions unexpected charges, agent investigates and resolves",
    "expected_coherence": True,
    "expected_retention": "excellent",
    "messages": [
        "I just got charged $29.99 on my account but I don't recognize this charge. Can you help?",
        "The charge date was December 1st. I checked and I didn't make any purchases that day.",
        "Oh yes! I forgot about the premium subscription. That makes sense. But I thought it was only $19.99?",
        "Okay, I see it now in my account settings. Thanks for clarifying. I was worried it was fraud!"
    ]
}

print(f"{'─'*70}")
print(f"Scenario: {scenario_billing['name']}")
print(f"Description: {scenario_billing['description']}")
print(f"Expected Retention: {scenario_billing['expected_retention']}")
print(f"{'─'*70}")

In [ ]:
# Run billing scenario
with mlflow.start_run(run_name=scenario_billing['name']) as run:
    conv_result_billing = agent.run_conversation(
        messages=scenario_billing['messages'],
        session_id=scenario_billing['session_id']
    )
    
    run_id_billing = run.info.run_id
    
    print(f"\nEvaluating session '{scenario_billing['session_id']}'...\n")
    
    try:
        eval_result_billing = agent.evaluate_session(scenario_billing['session_id'], run_id_billing)
        
        # Display results
        print(f"{'='*70}")
        print(f"Evaluation Results: {scenario_billing['name']}")
        print(f"{'='*70}")
        
        coherence_symbol = "✓" if eval_result_billing['coherence']['passed'] else "✗"
        coherence_label = "PASS" if eval_result_billing['coherence']['passed'] else "FAIL"
        print(f"\n📊 Coherence: {coherence_symbol} {coherence_label}")
        print(f"   Value: {eval_result_billing['coherence']['feedback_value']}")
        print(f"   Rationale: {eval_result_billing['coherence']['rationale']}")
        
        retention_value = str(eval_result_billing['context_retention']['feedback_value']).upper()
        retention_score = eval_result_billing['context_retention']['score']
        print(f"\n🧠 Context Retention: {retention_value}")
        print(f"   Score: {retention_score}/4")
        print(f"   Rationale: {eval_result_billing['context_retention']['rationale']}")
        
        print(f"\n{'='*70}\n")
        
    except Exception as e:
        print(f"\n✗ Evaluation failed: {e}")

## Additional Scenarios: Shipping Delay

Test with a shipping support conversation that demonstrates good (but not perfect) context retention.

In [ ]:
# Define shipping delay scenario
scenario_shipping = {
    "name": "Shipping Delay",
    "session_id": "session-shipping-004",
    "description": "Customer tracking delayed package, agent provides updates and resolution",
    "expected_coherence": True,
    "expected_retention": "good",
    "messages": [
        "My order #12345 was supposed to arrive yesterday but it still shows 'in transit'. What's going on?",
        "Yes, I checked the tracking number TRK789456123 and it hasn't updated since Monday.",
        "That would be helpful, thanks. The delivery address is 123 Main St, Apt 4B, Boston MA.",
        "Great! I appreciate you expediting it. Will I get an email when it ships out tomorrow?"
    ]
}

print(f"{'─'*70}")
print(f"Scenario: {scenario_shipping['name']}")
print(f"Description: {scenario_shipping['description']}")
print(f"Expected Retention: {scenario_shipping['expected_retention']}")
print(f"{'─'*70}")

In [ ]:
# Run shipping scenario
with mlflow.start_run(run_name=scenario_shipping['name']) as run:
    conv_result_shipping = agent.run_conversation(
        messages=scenario_shipping['messages'],
        session_id=scenario_shipping['session_id']
    )
    
    run_id_shipping = run.info.run_id
    
    print(f"\nEvaluating session '{scenario_shipping['session_id']}'...\n")
    
    try:
        eval_result_shipping = agent.evaluate_session(scenario_shipping['session_id'], run_id_shipping)
        
        # Display results
        print(f"{'='*70}")
        print(f"Evaluation Results: {scenario_shipping['name']}")
        print(f"{'='*70}")
        
        coherence_symbol = "✓" if eval_result_shipping['coherence']['passed'] else "✗"
        coherence_label = "PASS" if eval_result_shipping['coherence']['passed'] else "FAIL"
        print(f"\n📊 Coherence: {coherence_symbol} {coherence_label}")
        print(f"   Value: {eval_result_shipping['coherence']['feedback_value']}")
        print(f"   Rationale: {eval_result_shipping['coherence']['rationale']}")
        
        retention_value = str(eval_result_shipping['context_retention']['feedback_value']).upper()
        retention_score = eval_result_shipping['context_retention']['score']
        print(f"\n🧠 Context Retention: {retention_value}")
        print(f"   Score: {retention_score}/4")
        print(f"   Rationale: {eval_result_shipping['context_retention']['rationale']}")
        
        print(f"\n{'='*70}\n")
        
    except Exception as e:
        print(f"\n✗ Evaluation failed: {e}")

## 🎯 Try It Yourself!

Create and run your own custom conversation scenario using the helper function below.

In [ ]:
def run_custom_scenario(name: str, messages: List[str], session_id: str = None):
    """
    Helper function to run a complete conversation scenario.
    
    Args:
        name: Scenario name
        messages: List of user messages (turns)
        session_id: Optional session ID (auto-generated if not provided)
    
    Returns:
        Dictionary with conversation and evaluation results
    """
    import uuid
    
    if session_id is None:
        session_id = f"session-custom-{uuid.uuid4().hex[:8]}"
    
    print(f"\n{'='*70}")
    print(f"Running Custom Scenario: {name}")
    print(f"Session ID: {session_id}")
    print(f"Turns: {len(messages)}")
    print(f"{'='*70}\n")
    
    # Run conversation
    with mlflow.start_run(run_name=name) as run:
        conv_result = agent.run_conversation(
            messages=messages,
            session_id=session_id
        )
        
        run_id = run.info.run_id
        
        # Evaluate
        print(f"\nEvaluating session...\n")
        eval_result = agent.evaluate_session(session_id, run_id)
        
        # Display results
        print(f"{'='*70}")
        print(f"Evaluation Results: {name}")
        print(f"{'='*70}")
        
        coherence_symbol = "✓" if eval_result['coherence']['passed'] else "✗"
        coherence_label = "PASS" if eval_result['coherence']['passed'] else "FAIL"
        print(f"\n📊 Coherence: {coherence_symbol} {coherence_label}")
        print(f"   Rationale: {eval_result['coherence']['rationale']}")
        
        retention_value = str(eval_result['context_retention']['feedback_value']).upper()
        retention_score = eval_result['context_retention']['score']
        print(f"\n🧠 Context Retention: {retention_value} ({retention_score}/4)")
        print(f"   Rationale: {eval_result['context_retention']['rationale']}")
        
        print(f"\n{'='*70}\n")
        
        return {
            "conversation": conv_result,
            "evaluation": eval_result
        }

# Example: Uncomment and customize!
# result = run_custom_scenario(
#     name="Product Return Request",
#     messages=[
#         "I want to return a laptop I bought last week. It's not working properly.",
#         "The order number is ORD-9876. The screen keeps flickering.",
#         "Yes, I have the original box and all accessories.",
#         "Great! I'll ship it back tomorrow. Thanks for your help!"
#     ]
# )

print("✓ Helper function defined: run_custom_scenario()")
print("  Uncomment the example above to try it!")

## 🧪 Multi-Scenario Experiment

Run all scenarios in a batch and compare the evaluation results.

In [ ]:
# Collect all scenario results
all_results = []

# Note: We already ran scenarios above, so we'll use those results
# In a fresh run, you'd execute all scenarios here

try:
    all_results.append({
        "scenario": scenario1['name'],
        "session_id": scenario1['session_id'],
        "turns": len(scenario1['messages']),
        "coherence": "PASS" if eval_result1['coherence']['passed'] else "FAIL",
        "retention": eval_result1['context_retention']['feedback_value'],
        "retention_score": eval_result1['context_retention']['score']
    })
    
    all_results.append({
        "scenario": scenario2['name'],
        "session_id": scenario2['session_id'],
        "turns": len(scenario2['messages']),
        "coherence": "PASS" if eval_result2['coherence']['passed'] else "FAIL",
        "retention": eval_result2['context_retention']['feedback_value'],
        "retention_score": eval_result2['context_retention']['score']
    })
    
    all_results.append({
        "scenario": scenario_billing['name'],
        "session_id": scenario_billing['session_id'],
        "turns": len(scenario_billing['messages']),
        "coherence": "PASS" if eval_result_billing['coherence']['passed'] else "FAIL",
        "retention": eval_result_billing['context_retention']['feedback_value'],
        "retention_score": eval_result_billing['context_retention']['score']
    })
    
    all_results.append({
        "scenario": scenario_shipping['name'],
        "session_id": scenario_shipping['session_id'],
        "turns": len(scenario_shipping['messages']),
        "coherence": "PASS" if eval_result_shipping['coherence']['passed'] else "FAIL",
        "retention": eval_result_shipping['context_retention']['feedback_value'],
        "retention_score": eval_result_shipping['context_retention']['score']
    })
    
    print("✓ Collected results from all scenarios")
    
except Exception as e:
    print(f"⚠️  Some scenarios may not have run yet. Error: {e}")
    print("   Run all scenario cells above first.")

In [ ]:
# Display summary table
if len(all_results) > 0:
    results_df = pd.DataFrame(all_results)
    
    print("\n" + "="*70)
    print("📊 MULTI-SCENARIO EXPERIMENT SUMMARY")
    print("="*70)
    print(f"\nTotal Scenarios Run: {len(all_results)}")
    print(f"\n{results_df.to_string(index=False)}")
    
    # Calculate statistics
    avg_retention_score = results_df['retention_score'].mean()
    coherence_pass_rate = (results_df['coherence'] == 'PASS').sum() / len(results_df) * 100
    
    print(f"\n{'='*70}")
    print("📈 STATISTICS")
    print(f"{'='*70}")
    print(f"Average Context Retention Score: {avg_retention_score:.2f}/4")
    print(f"Coherence Pass Rate: {coherence_pass_rate:.0f}%")
    
    # Retention distribution
    retention_counts = results_df['retention'].value_counts()
    print(f"\nContext Retention Distribution:")
    for level, count in retention_counts.items():
        print(f"  {level}: {count} scenario(s)")
    
    print(f"\n{'='*70}\n")
else:
    print("No results to display. Run scenario cells above first.")

## Understanding the Mechanics

Let's examine how the session-level evaluation works under the hood.

In [ ]:
# Show the judge instructions with {{ conversation }} template
print("Coherence Judge Instructions:")
print("="*70)
print(get_coherence_judge_instructions())
print("\n" + "="*70 + "\n")

print("Context Retention Judge Instructions:")
print("="*70)
print(get_context_retention_judge_instructions())
print("\n" + "="*70 + "\n")

print("💡 Key Observation:")
print("Both judge instructions contain {{ conversation }} template variable.")
print("This automatically makes them session-level judges in MLflow 3.7!")

In [ ]:
# Show the session tracking pattern
print("Session Tracking Code Pattern:")
print("="*70)
print("""
@mlflow.trace(span_type=SpanType.CHAT_MODEL, name="handle_support_message")
def handle_message(self, message: str, session_id: str) -> str:
    # CRITICAL: Update trace with session metadata
    mlflow.update_current_trace(
        metadata={"mlflow.trace.session": session_id}
    )
    
    # ... rest of message handling ...
    
    return assistant_message
""")
print("="*70)
print("\n💡 This simple metadata update enables all of MLflow 3.7's session-level features!")
print("   - Groups multiple traces into one session")
print("   - Enables {{ conversation }} template substitution")
print("   - Allows session-level evaluation instead of turn-level")

## 📚 Key Concepts Summary

### MLflow 3.7 Session-Level Evaluation Features

1. **Session Tracking**: `mlflow.update_current_trace(metadata={'mlflow.trace.session': session_id})`
   - Tags each trace with a session ID
   - Enables grouping of related traces
   - Critical for multi-turn conversation tracking

2. **{{ conversation }} Template**: Automatically aggregates session traces
   - Used in judge instructions
   - Makes judges automatically session-level aware
   - MLflow substitutes with formatted conversation transcript

3. **Session-Level Judges**: Two types demonstrated
   - **Boolean Judge** (Coherence): Returns True/False
   - **Categorical Judge** (Context Retention): Returns excellent/good/fair/poor

4. **Session-Level Evaluation**: `mlflow.genai.evaluate()` with session-level judges
   - Evaluates entire conversation, not individual turns
   - Returns one result per session (not per trace)
   - Enables assessment of coherence, context retention, and flow

### Workflow Summary

```
1. Setup MLflow tracking
2. Initialize CustomerSupportAgent with judges
3. Run multi-turn conversation (each turn creates a trace)
4. Each trace tagged with session ID via mlflow.update_current_trace()
5. Evaluate session with mlflow.genai.evaluate()
6. Judges receive {{ conversation }} with full transcript
7. Get coherence and context retention results
```

### Judge Configuration

- **Coherence Judge**: `feedback_value_type=bool`
- **Context Retention Judge**: `feedback_value_type=Literal["excellent", "good", "fair", "poor"]`
- Both use `{{ conversation }}` template → automatically session-level
- Both return feedback_value + rationale

## 📊 Context Retention Levels Explained

The context retention judge evaluates conversations on a 4-level scale:

| Level | Score | Description |
|-------|-------|-------------|
| **Excellent** | 4 | Perfectly recalls all relevant prior context; seamlessly references earlier information; builds on previous exchanges without repetition |
| **Good** | 3 | Recalls most relevant context; occasionally references earlier info; minor lapses but overall maintains context |
| **Fair** | 2 | Some context retention issues; asks redundant questions; doesn't fully leverage earlier information |
| **Poor** | 1 | Frequently forgets prior context; treats each turn independently; significant repetition or contradictions |

### Examples from Our Scenarios:

- **Printer Troubleshooting**: Expected "excellent" - Agent remembers HP LaserJet 3000, power cable checks, outlet testing
- **Account Access**: Expected "good" - Agent tracks password issue, email address, but may have minor inefficiencies
- **Billing Inquiry**: Expected "excellent" - Agent remembers $29.99 charge, Dec 1st date, premium subscription, price change
- **Shipping Delay**: Expected "good" - Agent tracks order #12345, tracking number, address, but might ask for info already in system

## 🚀 Next Steps

1. **Explore MLflow UI**: Run `mlflow ui` to see detailed traces
   - View session groupings
   - Explore trace hierarchies
   - Compare evaluation results across sessions

2. **Customize Judge Instructions**: Edit `prompts.py` to change evaluation criteria
   - Modify coherence requirements
   - Adjust context retention levels
   - Add new evaluation dimensions

3. **Try Different Models**: Experiment with different agent and judge models
   - Compare OpenAI vs Databricks models
   - Test various temperature settings
   - Evaluate model performance on context retention

4. **Create Custom Scenarios**: Design your own conversation tests
   - Test edge cases (very long conversations, context switching)
   - Domain-specific scenarios (e.g., healthcare, finance)
   - Multi-language support

5. **Apply to Your Use Case**: Adapt this pattern for your own applications
   - Customer support bots
   - Virtual assistants
   - Educational tutors
   - Any multi-turn conversational AI

## 📖 Resources

- [MLflow GenAI Judges Documentation](https://mlflow.org/docs/latest/llms/llm-evaluate/index.html)
- [MLflow 3.7 Release Notes](https://github.com/mlflow/mlflow/releases/tag/v3.7.0)
- [Session-Level Evaluation Guide](https://mlflow.org/docs/latest/llms/tracing/index.html)
- [Customer Support Agent README](README.md)
- [Agent Planning Judge Tutorial](../agent_planning/agent_planning_judge.ipynb) - Related pattern